In [10]:
# -*- coding: utf-8 -*-
"""Constructing_model.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/14Jtjr_GEx3PuffgxYoe8Ktrbd0skq-p-
"""

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

PATH = '/content/gdrive/MyDrive/Colab Notebooks/Pneumonia Prediction/chest_xray'

import os

for folder in ['train','test','val']:
  for t in ['NORMAL','PNEUMONIA']:
    print(f'{folder}: {t} count:{len(os.listdir(os.path.join(PATH,folder,t)))}')

import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import numpy as np
import pandas as pd

labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

train = get_data(PATH+'/train')
test = get_data(PATH+'/test')
val = get_data(PATH+'/val')

x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

x_test[0].shape

type(y_train)
y_train.shape

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
y_val = y_val.reshape(-1,1)

datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range = 30,  
        zoom_range = 0.2,  
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip = True,
        vertical_flip=False)


datagen.fit(x_train)

model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.summary()

model.compile(optimizer = "rmsprop" ,loss = 'binary_crossentropy' ,metrics = ['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience = 2, 
                                            verbose=1,
                                            factor=0.3, 
                                            min_lr=0.000001)

Mounted at /content/gdrive/
train: NORMAL count:1341
train: PNEUMONIA count:3875
test: NORMAL count:234
test: PNEUMONIA count:390
val: NORMAL count:8
val: PNEUMONIA count:8


<ipython-input-10-231ac8938f3f>:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 150, 150, 32)      320       
                                                                 
 batch_normalization_10 (Bat  (None, 150, 150, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 dropout_8 (Dropout)         (None, 75, 75, 64)        0         
                                                                 
 batch_normalization_11 (Bat  (None, 75, 75, 64)      

In [11]:
from sklearn.utils import class_weight
import numpy as np
corpus = []
corpus.extend(['NORMAL']*1341)
corpus.extend(['PNEUMONIA']*3875)
class_weights = class_weight.compute_class_weight('balanced',classes =['NORMAL','PNEUMONIA'],y = corpus)
class_weights

array([1.9448173 , 0.67303226])

In [12]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 32), epochs = 10, validation_data = datagen.flow(x_val, y_val),
                    callbacks = learning_rate_reduction,class_weight = {0:1.9448173,1:0.67303226})

print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

model.evaluate(x_val,y_val)[0]

Epoch 1/10
163/163 [==============================] - 17s 83ms/step - loss: 0.9787 - accuracy: 0.8299 - val_loss: 11.7802 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/10
163/163 [==============================] - 14s 83ms/step - loss: 0.2828 - accuracy: 0.8942 - val_loss: 14.7838 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/10
163/163 [==============================] - ETA: 0s - loss: 0.2325 - accuracy: 0.9105
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
163/163 [==============================] - 14s 85ms/step - loss: 0.2325 - accuracy: 0.9105 - val_loss: 15.2446 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/10
163/163 [==============================] - 13s 82ms/step - loss: 0.1751 - accuracy: 0.9325 - val_loss: 14.4861 - val_accuracy: 0.5000 - lr: 3.0000e-04
Epoch 5/10
163/163 [==============================] - ETA: 0s - loss: 0.1545 - accuracy: 0.9411
Epoch 5: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
163/163 [======================

0.37563809752464294

In [48]:
predictions = model.predict(x_test)
print(type(predictions.flatten()))
predictions = list(map(lambda x: 1 if x > 0.33 else 0,predictions.flatten()))

20/20 [==============================] - 0s 9ms/step
<class 'numpy.ndarray'>


In [49]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

confusion_matrix(predictions,y_test)

array([[358,  24],
       [ 32, 210]])

In [50]:
accuracy_score(predictions,y_test)

0.9102564102564102

In [51]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       382
           1       0.90      0.87      0.88       242

    accuracy                           0.91       624
   macro avg       0.91      0.90      0.90       624
weighted avg       0.91      0.91      0.91       624



In [55]:
model.save(PATH + '/little_imporved_model.h5')

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [50]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.models import load_model 
import numpy as np
import cv2


PATH = '/content/gdrive/MyDrive/Colab Notebooks/Pneumonia Prediction/chest_xray'

model = load_model(PATH + '/little_imporved_model.h5')

def predict_from_path(img_path):
  img = load_img(img_path,target_size = (150,150),color_mode = 'grayscale')
  img = img_to_array(img)
  img /= 255.0
  img = np.expand_dims(img,axis = 0)
  return model.predict(img)[0][0]

def predict_from_img(img):
  img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  img = img/255.0
  img = np.expand_dims(img,axis = 0)
  output = model.predict(img)[0][0]
  return {'NORMAL':float(output),'PNEUMONIA':float(1-output)}

In [ ]:
!pip install gradio

In [25]:
predict_from_path('/content/gdrive/MyDrive/Colab Notebooks/Pneumonia Prediction/chest_xray/test/NORMAL/IM-0001-0001.jpeg')

1/1 [==============================] - 0s 288ms/step


0.9920779

In [47]:
import gradio as gr
image = gr.inputs.Image(shape=(150,150))
label = gr.outputs.Label(num_top_classes=2)
gr.Interface(fn=predict_from_img, inputs=image, outputs=label).launch(share=True)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4ef289275ec212338a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [49]:
import keras
import tensorflow as tf
print(tf.__version__)
print(np.__version__)
print(cv2.__version__)

2.12.0
1.22.4
4.7.0
